In [21]:
import datetime

import drms
import astropy.units as u
from sunpy.net import Fido, attrs as a
from sunpy.map import Map

In [2]:
tr = a.Time("2011-02-13 00:00:00.000", "2011-02-16 00:00:00.000")
result = Fido.search(tr,
                      a.hek.FL,
                      a.hek.FL.GOESCls > "M1.0",
                      a.hek.OBS.Observatory == "GOES")

In [3]:
hek_results = result["hek"]["event_starttime", "event_peaktime",
                             "event_endtime", "fl_goescls", "ar_noaanum"]
hek_results

event_starttime,event_peaktime,event_endtime,fl_goescls,ar_noaanum
Time,Time,Time,str4,int64
2011-02-13 17:28:00.000,2011-02-13 17:38:00.000,2011-02-13 17:47:00.000,M6.6,11158
2011-02-14 17:20:00.000,2011-02-14 17:26:00.000,2011-02-14 17:32:00.000,M2.2,11158
2011-02-15 01:44:00.000,2011-02-15 01:56:00.000,2011-02-15 02:06:00.000,X2.2,11158


In [4]:
peaktime= hek_results["event_peaktime"]
peaktime

<Time object: scale='utc' format='iso' value=['2011-02-13 17:38:00.000' '2011-02-14 17:26:00.000'
 '2011-02-15 01:56:00.000']>

In [5]:
times = []

time_interval = datetime.timedelta(minutes=30)

for pt in peaktime:
    times.append(a.Time(pt - time_interval, pt + time_interval))

In [6]:
times

[<sunpy.net.attrs.Time(2011-02-13 17:08:00.000, 2011-02-13 18:08:00.000)>,
 <sunpy.net.attrs.Time(2011-02-14 16:56:00.000, 2011-02-14 17:56:00.000)>,
 <sunpy.net.attrs.Time(2011-02-15 01:26:00.000, 2011-02-15 02:26:00.000)>]

# SHARP

In [7]:
c = drms.Client(email="mgjeon@khu.ac.kr")

In [8]:
c.info("hmi.sharp_cea_720s").segments

,type,units,protocol,dims,note
name,,,,,
magnetogram,int,Gauss,fits,VARxVAR,Line-of-sight magnetogram in CEA projection
bitmap,char,Enumerated,fits,VARxVAR,Mask for the patch in CEA coordinates
Dopplergram,int,m/s,fits,VARxVAR,Dopplergram in CEA projection
continuum,int,DN/s,fits,VARxVAR,Intensitygram in CEA projection
Bp,int,Gauss,fits,VARxVAR,"B_phi, positive westward"
Bt,int,Gauss,fits,VARxVAR,"B_theta, positive southward"
Br,int,Gauss,fits,VARxVAR,"B_r, positive up"
Bp_err,int,Gauss,fits,VARxVAR,Standard deviation of B_phi
Bt_err,int,Gauss,fits,VARxVAR,Standard deviation of B_theta


In [14]:
c.pkeys("hmi.sharp_cea_720s")

['HARPNUM', 'T_REC']

In [15]:
find_ar = c.query("hmi.Mharp_720s[][%sZ]" % peaktime[0],
                   key = ["NOAA_AR", "HARPNUM"])
find_ar

,NOAA_AR,HARPNUM
0,11157,364
1,11156,367
2,0,369
3,11159,371
4,0,372
5,11158,377
6,0,378
7,0,380
8,0,381
9,11160,384


In [56]:
hmi_results = Fido.search(
    tr,
    a.jsoc.Notify("mgjeon@khu.ac.kr"),
    a.jsoc.Series("hmi.sharp_cea_720s"),
    a.jsoc.PrimeKey('HARPNUM', '377'),
    a.jsoc.Segment("Bp") & a.jsoc.Segment("Bt") & a.jsoc.Segment("Br"),
)

In [ ]:
hmi_files = Fido.fetch(hmi_results, path="download/11158/hmi/{file}")

# AIA

In [19]:
c.info("aia.lev1_euv_12s").segments

,type,units,protocol,dims,note
name,,,,,
image,int,None,link via lev1,None,AIA level 1 image
spikes,int,None,link via lev1,None,Cosmic ray information


In [20]:
c.pkeys("aia.lev1_euv_12s")

['T_REC', 'WAVELNTH']

In [51]:
wavelengths = [94, 
               131, 
               171, 
               193, 
               211, 
               304, 
               335]

In [53]:
aia_results = {}

for w in wavelengths:
    aia_results[str(w)] = Fido.search(
        times[0] | times[1] | times[2],
        a.jsoc.Notify("mgjeon@khu.ac.kr"),
        a.jsoc.Series("aia.lev1_euv_12s"),
        a.jsoc.Segment("image"),
        a.Wavelength(w*u.angstrom))

In [54]:
for key, value in aia_results.items():
    print(key, len(value))

94 3
131 3
171 3
193 3
211 3
304 3
335 3


In [ ]:
for wavelnth, result in aia_results.items():
    Fido.fetch(result, path=f"download/11158/aia/{wavelnth}/" + "{file}")